# Installation

In [3]:
!pip install -q torch
!pip install -q transformers
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain
!pip install -q langchain-community
!python -m pip install pypdf
!pip install -U langchain-openai

!pip install -q langchain_google_genai
!pip install chromadb langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 26.3 MB/s eta 0:00:

# Imports


In [4]:
import os
import pandas as pd
from urllib.request import urlretrieve
from langchain_community.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline, OpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import LLMChain, RetrievalQA, create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings



ModuleNotFoundError: No module named 'groq'

In [5]:
from dataclasses import dataclass
import bs4
import warnings

# Get and Prepare Data

In [6]:
df = pd.read_csv('/content/final_cocktails.csv')
df.head()

,Unnamed: 0,id,name,alcoholic,category,glassType,instructions,drinkThumbnail,ingredients,ingredientMeasures,text
0,0,0,A1,Alcoholic,Cocktail,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grand Marnier', 'Lemon Juice', 'Grena...","['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Sh...",question Generate a cocktail with Gin Grand Ma...
1,1,1,ABC,Alcoholic,Shot,Shot glass,Layered in a shot glass.,https://www.thecocktaildb.com/images/media/dri...,"['Amaretto', 'Baileys irish cream', 'Cognac']","['1/3 ', '1/3 ', '1/3 ']",question Generate a cocktail with Amaretto Bai...
2,2,2,Ace,Alcoholic,Cocktail,Martini Glass,Shake all the ingredients in a cocktail shaker...,https://www.thecocktaildb.com/images/media/dri...,"['Gin', 'Grenadine', 'Heavy cream', 'Milk', 'E...","['2 shots ', '1/2 shot ', '1/2 shot ', '1/2 sh...",question Generate a cocktail with Gin Grenadin...
3,3,3,ACID,Alcoholic,Shot,Shot glass,Poor in the 151 first followed by the 101 serv...,https://www.thecocktaildb.com/images/media/dri...,"['151 proof rum', 'Wild Turkey']","['1 oz Bacardi ', '1 oz ']",question Generate a cocktail with 151 proof ru...
4,4,4,Adam,Alcoholic,Ordinary Drink,Cocktail glass,"In a shaker half-filled with ice cubes, combin...",https://www.thecocktaildb.com/images/media/dri...,"['Dark rum', 'Lemon juice', 'Grenadine']","['2 oz ', '1 oz ', '1 tsp ']",question Generate a cocktail with Dark rum Lem...


In [7]:
print(df.columns)


Index(['Unnamed: 0', 'id', 'name', 'alcoholic', 'category', 'glassType',
       'instructions', 'drinkThumbnail', 'ingredients', 'ingredientMeasures',
       'text'],
      dtype='object')


In [8]:
df['combined_text'] = df.apply(
    lambda row: f"Name: {row['name']}. "
                f"Alcoholic: {row['alcoholic']}. "
                f"Category: {row['category']}. "
                f"Glass Type: {row['glassType']}. "
                f"Ingredients: {row['ingredients']} "
                f"({row['ingredientMeasures']}). "
                f"Instructions: {row['instructions']}.",
    axis=1
)

In [9]:
df['combined_text'][0]

"Name: A1. Alcoholic: Alcoholic. Category: Cocktail. Glass Type: Cocktail glass. Ingredients: ['Gin', 'Grand Marnier', 'Lemon Juice', 'Grenadine'] (['1 3/4 shot ', '1 Shot ', '1/4 Shot', '1/8 Shot']). Instructions: Pour all ingredients into a cocktail shaker, mix and serve over ice into a chilled glass.."

In [10]:
docs = [
    Document(
        page_content=row['combined_text'],
        metadata={
            'name': row['name'],
            'alcoholic': row['alcoholic'],
            'category': row['category'],
            'glassType': row['glassType'],
            'ingredients': row['ingredients'],
            'ingredientMeasures': row['ingredientMeasures'],
            'instructions': row['instructions']
        }
    )
    for _, row in df.iterrows()
]

In [11]:
len(docs)

425

In [12]:
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')


<ipython-input-12-3f763d9c26c3>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Create the embeddings + retriever


In [13]:
db = FAISS.from_documents(docs, embeddings)

# Setup the LLM chain

In [14]:
retriever = db.as_retriever()


In [15]:
system_prompt = (
"""
You are an experienced bartender with years of expertise in crafting cocktails and advising clients.

Your task is to provide cocktail recommendations based on the user’s preferences and to answer any questions about cocktails accurately and concisely.
If the information about the user’s preferences or other details is insufficient to provide a high-quality response, you must ask for clarification.
You also take into account the user’s previous preferences and responses, if provided earlier. If the user has already mentioned their favorite ingredients, you use this information in your response.

**Response Guidelines:**
1. Answer concisely but with enough detail to ensure clarity.
2. Only ask for information that is genuinely necessary to provide a quality recommendation or response.
3. Your tone should be professional, friendly, and confident.

Use the following context to help:
{context}
"""
)

In [16]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [46]:
llm = OpenAI(openai_api_key = 'your_key')


In [18]:
question_answering_chain=create_stuff_documents_chain(llm, chat_prompt)

In [19]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [20]:
response = rag_chain.invoke({"input":"What are the 5 cocktails containing lemon?"})

In [21]:
response["answer"]

"\n\nBartender: There are numerous cocktails that contain lemon as an ingredient, but some popular ones are the Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and the classic Tom Collins. Is there a specific type of cocktail you're interested in?"

In [22]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [23]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [24]:
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

In [25]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [26]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [27]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [28]:
chat_history = []

In [29]:
question1 = "What are the 5 cocktails containing lemon?"

In [30]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [31]:
message1["answer"]

'\n\nBot: The five cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Gin and Tonic. Is there a specific cocktail you are interested in learning more about?'

In [32]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [33]:
chat_history


[HumanMessage(content='What are the 5 cocktails containing lemon?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n\nBot: The five cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Gin and Tonic. Is there a specific cocktail you are interested in learning more about?', additional_kwargs={}, response_metadata={})]

In [34]:
second_question = "Write a recipe for any of them"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])


Bot: For the Lemon Drop cocktail, you will need 1 1/2 oz vodka, 1/2 oz triple sec, 3/4 oz lemon juice, and 1 tsp superfine sugar. Shake all ingredients with ice and strain into a chilled cocktail glass rimmed with sugar. Garnish with a lemon twist.


In [35]:
store = {}

In [36]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [37]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [38]:
conversational_rag_chain.invoke(
    {"input": "What are the 5 cocktails containing lemon?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'\n\nResponse: Some popular cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Tom Collins.'

In [39]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What are the 5 cocktails containing lemon?', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\nResponse: Some popular cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Tom Collins.', additional_kwargs={}, response_metadata={})])}

In [40]:
conversational_rag_chain.invoke(
    {"input": "Write a recipe for any of them"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'\nAI: Here is a recipe for the Lemon Drop cocktail:\n\nIngredients:\n- 1 1/2 shots of vodka\n- 1 1/2 shots of Cointreau\n- Juice of 1 wedge of lemon\n- Sugar for rimming the glass\n- Ice\n\nInstructions:\n1. In a shaker, combine vodka, Cointreau, and lemon juice.\n2. Add ice and shake well.\n3. Rim a chilled cocktail glass with sugar.\n4. Strain the cocktail into the glass.\n5. Garnish with a lemon wedge.\n6. Serve and enjoy your refreshing Lemon Drop cocktail!'

In [41]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What are the 5 cocktails containing lemon?

AI: 

Response: Some popular cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Tom Collins.

User: Write a recipe for any of them

AI: 
AI: Here is a recipe for the Lemon Drop cocktail:

Ingredients:
- 1 1/2 shots of vodka
- 1 1/2 shots of Cointreau
- Juice of 1 wedge of lemon
- Sugar for rimming the glass
- Ice

Instructions:
1. In a shaker, combine vodka, Cointreau, and lemon juice.
2. Add ice and shake well.
3. Rim a chilled cocktail glass with sugar.
4. Strain the cocktail into the glass.
5. Garnish with a lemon wedge.
6. Serve and enjoy your refreshing Lemon Drop cocktail!



In [42]:
conversational_rag_chain.invoke(
    {"input": "What is a recipe of Lemon Shot?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'\nAI: Here is a recipe for Lemon Shot:\n\nIngredients:\n- 1/2 oz of Galliano\n- 1/2 oz of Absolut Citron\n- 1 lemon wedge\n- Sugar for sprinkling\n- 151 proof rum\n- Ice\n\nInstructions:\n1. In a shot glass, mix Galliano and Absolut Citron.\n2. Sprinkle sugar over a lemon wedge and place it on top of the shot glass.\n3. Pour 151 proof rum over the lemon wedge and glass.\n4. Light the rum with a lighter and let it burn for a second.\n5. Do the shot quickly and suck on the lemon wedge.\n6. Enjoy the taste of sweet lemonade!'

In [43]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What are the 5 cocktails containing lemon?

AI: 

Response: Some popular cocktails that contain lemon are Lemon Drop, New York Lemonade, Gin Lemon, Vodka Lemon, and Tom Collins.

User: Write a recipe for any of them

AI: 
AI: Here is a recipe for the Lemon Drop cocktail:

Ingredients:
- 1 1/2 shots of vodka
- 1 1/2 shots of Cointreau
- Juice of 1 wedge of lemon
- Sugar for rimming the glass
- Ice

Instructions:
1. In a shaker, combine vodka, Cointreau, and lemon juice.
2. Add ice and shake well.
3. Rim a chilled cocktail glass with sugar.
4. Strain the cocktail into the glass.
5. Garnish with a lemon wedge.
6. Serve and enjoy your refreshing Lemon Drop cocktail!

User: What is a recipe of Lemon Shot?

AI: 
AI: Here is a recipe for Lemon Shot:

Ingredients:
- 1/2 oz of Galliano
- 1/2 oz of Absolut Citron
- 1 lemon wedge
- Sugar for sprinkling
- 151 proof rum
- Ice

Instructions:
1. In a shot glass, mix Galliano and Absolut Citron.
2. Sprinkle sugar over a lemon wedge and place it 

# ІНТЕРФЕЙС

In [44]:
!pip install gradio

In [45]:
import gradio as gr
from langchain.schema import AIMessage, HumanMessage

retriever = db.as_retriever()

system_prompt = (
    """
    You are an experienced bartender with years of expertise in crafting cocktails and advising clients.

    Your task is to provide cocktail recommendations based on the user’s preferences and to answer any questions about cocktails accurately and concisely.
    If the information about the user’s preferences or other details is insufficient to provide a high-quality response, you must ask for clarification.
    You also take into account the user’s previous preferences and responses, if provided earlier. If the user has already mentioned their favorite ingredients, you use this information in your response.

    **Response Guidelines:**
    1. Answer concisely but with enough detail to ensure clarity.
    2. Only ask for information that is genuinely necessary to provide a quality recommendation or response.
    3. Your tone should be professional, friendly, and confident.

    Use the following context to help:
    {context}
    """
)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

llm = OpenAI(openai_api_key="your_key")

question_answering_chain = create_stuff_documents_chain(llm, chat_prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

def chat_with_bot(user_input, session_id):
    try:
        response = conversational_rag_chain.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}},
        )

        history = store[session_id].messages
        messages = []
        for message in history:
            if isinstance(message, AIMessage):
                messages.append(["AI", message.content])
            else:
                messages.append(["User", message.content])

        return messages
    except Exception as e:
        return [["Error", str(e)]]

with gr.Blocks() as demo:
    gr.Markdown("# Чат-бот RAG з пам'яттю")

    session_id = gr.Textbox(label="ID сесії", placeholder="Унікальний ідентифікатор сесії", value="default_session")
    user_input = gr.Textbox(label="Введіть ваш запит", placeholder="Напишіть щось...")
    chat_history = gr.Chatbot(label="Історія чату")
    submit_button = gr.Button("Відправити")

    def process_input(user_input, session_id):
        messages = chat_with_bot(user_input, session_id)
        return messages

    submit_button.click(
        fn=process_input,
        inputs=[user_input, session_id],
        outputs=[chat_history],
    )

demo.launch()


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a92ebdb9577ce6bd00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
